In [101]:
import os

from PIL import Image
Image.MAX_IMAGE_PIXELS=None
import numpy as np
from skimage import measure
from copy import deepcopy
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from glob import glob

In [102]:
from ctypes.util import *
from ctypes import *
find_library("libopenslide-0.dll")
lib = cdll.LoadLibrary(find_library("libopenslide-0.dll"))
import openslide

In [117]:
# open both wsi and annotation map
wsi_src = r'\\fatherserverdw\kyuex\great'
wsipaths = glob(os.path.join(wsi_src,'*ndpi'))
iter_order = [2,10,5,4,6,11,7,9,8,12,3,1]

In [117]:
#todo:add class id and track area of each class
#add maximum overlapp


In [118]:
for wsipath in wsipaths[0:1]:
    wsi = openslide.OpenSlide(wsipath)
    annotation_pth = wsipath.replace('ndpi','png')
    annotation = Image.open(annotation_pth)  #40X resolution

    #create folders for mskcrop and imcrop
    folder_name = os.path.basename(wsipath)
    folder_name = folder_name.removesuffix('.ndpi')
    dirName1 = r'\\fatherserverdw\kyuex\great\mask and crop\{}imcrop'.format(folder_name)
    dirName2 = r'\\fatherserverdw\kyuex\great\mask and crop\{}mskcrop'.format(folder_name)
    if not os.path.exists(dirName1):
        os.mkdir(dirName1)
    if not os.path.exists(dirName2):
        os.mkdir(dirName2)

    #resize wsi image
    target_level = 2
    rsf = wsi.level_downsamples[target_level]
    target_dim = wsi.level_dimensions[target_level]
    annotation_resized = annotation.resize(target_dim,resample=0) #this step is a bottleneck
    annotation_resized_arr = np.array(annotation_resized)

    for classid in iter_order:
    #get only the region with the particular class id
        class_annotation = annotation_resized_arr == classid
        labeled_map = measure.label(class_annotation)
        regionprops = measure.regionprops(labeled_map)
        boundboxes = [_.bbox for _ in regionprops]

        for objID, boundbox in enumerate(boundboxes):
            #crop at the boundaries
            objmask =  deepcopy(labeled_map)
            objmask[objmask != objID + 1] = 0
            objmask[objmask == objID + 1]= classid#this field was class id
            objmask = objmask[boundbox[0]:boundbox[2], boundbox[1]:boundbox[3]]
            h, w = boundbox[2] - boundbox[0], boundbox[3] - boundbox[1]
            #read cropped region
            imcrop = wsi.read_region(location=(round(boundbox[1]*rsf), round(boundbox[0]*rsf)), level=target_level, size=(w, h))
            imcrop_masked = np.multiply(np.array(imcrop),objmask[..., None].astype(np.bool))


            Image.fromarray((imcrop_masked).astype(np.uint8)).save(r'\\fatherserverdw\kyuex\great\mask and crop\{}imcrop\class{}_obj{}.png'.format(folder_name,classid, objID))
            Image.fromarray(objmask.astype(np.uint8)).save(r'\\fatherserverdw\kyuex\great\mask and crop\{}mskcrop\classnew{}_obj{}.png'.format(folder_name,classid, objID))


C:\Users\kyuha\AppData\Local\Temp\ipykernel_5488\1101146299.py:39: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  imcrop_masked = np.multiply(np.array(imcrop),objmask[..., None].astype(np.bool))


In [108]:
annotation.size

(74880, 36608)

In [109]:
annotation_resized.size

(18720, 9152)

In [ ]:
#----end,test code-------

In [112]:
wsipath = wsipaths[0]
#anno_src = os.path.expanduser(r'\\Users\kyuha\PycharmProjects\wsi_analysis\ashley')
#anno_fn = '1C1temp.png'
#annotation_map_path = os.path.join(anno_src, anno_fn)
wsi = openslide.OpenSlide(wsipath)
annotation_pth = wsipath.replace('ndpi','png')
annotation = Image.open(annotation_pth)  #40X resolution


In [113]:
folder_name = os.path.basename(wsipath)
folder_name = folder_name.removesuffix('.ndpi')

In [115]:
target_level = 2
rsf = wsi.level_downsamples[target_level]
target_dim = wsi.level_dimensions[target_level]
annotation_resized = annotation.resize(target_dim,resample=0) #this step is a bottleneck
annotation_resized_arr = np.array(annotation_resized)

C:\Users\kyuha\AppData\Local\Temp\ipykernel_5488\253837825.py:4: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  annotation_resized = annotation.resize(target_dim,resample=Image.BICUBIC) #this step is a bottleneck


In [116]:

#labeled_map = measure.label(annotation_resized_arr)
for classid in iter_order:
    #get only the region with the particular class id
    class_annotation = annotation_resized_arr == classid
    labeled_map = measure.label(class_annotation)
    regionprops = measure.regionprops(labeled_map)
    boundboxes = [_.bbox for _ in regionprops]

    for objID, boundbox in enumerate(boundboxes):
        #crop at the boundaries
        objmask =  deepcopy(labeled_map)
        objmask[objmask != objID + 1] = 0
        objmask[objmask == objID + 1]= classid#this field was class id
        objmask = objmask[boundbox[0]:boundbox[2], boundbox[1]:boundbox[3]]
        h, w = boundbox[2] - boundbox[0], boundbox[3] - boundbox[1]
        #read cropped region
        imcrop = wsi.read_region(location=(round(boundbox[1]*rsf), round(boundbox[0]*rsf)), level=target_level, size=(w, h))
        imcrop_masked = np.multiply(np.array(imcrop),objmask[..., None].astype(np.bool))

        #create two folders (for crop and mask)
        dirName1 = r'\\fatherserverdw\kyuex\great\mask and crop\{}imcrop'.format(folder_name)
        dirName2 = r'\\fatherserverdw\kyuex\great\mask and crop\{}mskcrop'.format(folder_name)

        if not os.path.exists(dirName1):
            os.mkdir(dirName1)

        if not os.path.exists(dirName2):
            os.mkdir(dirName2)

        Image.fromarray((imcrop_masked).astype(np.uint8)).save(r'\\fatherserverdw\kyuex\great\mask and crop\{}imcrop\class{}_obj{}.png'.format(folder_name,classid, objID))
        Image.fromarray(objmask.astype(np.uint8)).save(r'\\fatherserverdw\kyuex\great\mask and crop\{}mskcrop\classnew{}_obj{}.png'.format(folder_name,classid, objID))

C:\Users\kyuha\AppData\Local\Temp\ipykernel_5488\1725907988.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  imcrop_masked = np.multiply(np.array(imcrop),objmask[..., None].astype(np.bool))


KeyboardInterrupt: 

In [10]:
#---unedited---

In [13]:
#this code is just for only one object in each class, don't use
labeled_map = measure.label(annotation_resized_arr.astype(np.bool))
regionprops = measure.regionprops(labeled_map)
boundboxes = [_.bbox for _ in regionprops]
for objID, boundbox in enumerate(boundboxes): #iterate each connected object on deeplab mask
    connected_obj_mask = labeled_map == objID +1
    dlmask = np.multiply(annotation_resized_arr,connected_obj_mask)
    dlmask = dlmask[boundbox[0]:boundbox[2], boundbox[1]:boundbox[3]]
    h, w = boundbox[2] - boundbox[0], boundbox[3] - boundbox[1]
    imcrop = wsi.read_region(location=(round(boundbox[1]*rsf), round(boundbox[0]*rsf)), level=target_level, size=(w, h))
    imcrop_masked = np.multiply(np.array(imcrop),dlmask[..., None].astype(np.bool))
    Image.fromarray((imcrop_masked).astype(np.uint8)).save('imcrop/obj{}.png'.format(objID))
    Image.fromarray(dlmask.astype(np.uint8)).save('mskcrop/obj{}.png'.format(objID))

2

In [ ]:

# iterate each class and each annotation object to make a crop
# for classid in iter_order:
#     annotation2crop = np.zeros_like(annotation)
#     #all annotation for the current class id
#     annotation2crop[np.array(annotation) == classid] = 1
#     labeled_map = measure.label(annotation2crop)
#     #regionprops has properties of the label
#     regionprops = measure.regionprops(labeled_map)
#     #bounboxes is a tuple, (min_row, min_col, max_row, max_col)
#     boundboxes = list()
#     for region in regionprops:
#         boundboxes.append(region.bbox)
#
#     for objID, boundbox in enumerate(boundboxes):
#         #crop at the boundaries
#         objmask = deepcopy(labeled_map)
#         objmask[objmask != objID + 1] = 0
#         objmask[objmask == objID + 1] = 1
#
#         objmask = objmask[boundbox[0]:boundbox[2], boundbox[1]:boundbox[3]]
#         h, w = boundbox[2] - boundbox[0], boundbox[3] - boundbox[1]
#
#         #read cropped region
#         imcrop = wsi.read_region(location=(boundbox[1], boundbox[0]), level=target_level, size=(w, h)).convert(
#             'RGB')  #use level 2, more zoomed in
#         imcrop = np.array(imcrop)
#
#         #create mask to crop out the extra space around the annotation
#         objmask = deepcopy(labeled_map)
#         objmask[objmask != 1] = 0
#         objmask[objmask == 1] = 1
#         #boundbox = [round(_/10) for _ in boundbox] #downsize by 10
#         objmask2 = objmask[boundbox[0]:boundbox[2],
#                    boundbox[1]:boundbox[3]]  #crop out the objmask2, typecast to integer
#
#         #enlarge objmask2 by 10 to get objmask3, take transpose of objmask3 to get objmask4
#         #new_shape = list(objmask2.shape)
#         #new_shape = [_*10 for _ in new_shape]
#         #objmask3 = cv2.resize(objmask2.astype(np.float),tuple(new_shape), interpolation = cv2.INTER_NEAREST)#NEAREST preserves pixel value
#
#         #find boundary of objmask3, use cv2 contour
#         #this gives you a list of bounding pixels\
#         #objmask3_img = Image.fromarray(objmask3.astype(np.uint8))
#
#         #objmask3_array = np.asarray(objmask3_img)
#
#         #this gives me a list of tuples
#         #contour, hierarchy = cv2.findContours(objmask3_array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#
#         #loop through each pixel in bound_objmask3
#         #contour = np.array(contour)
#         #contour = contour.squeeze()
#
#         #for count,pix1 in enumerate(contour):
#         # while(count < len(contour) - 2): #is this right??
#         # for i in range(1, 5): #set n = 2 (split into two equal parts)
#         #   a = float((i) / 5)        # rescale 0 < i < n --> 0 < a < 1
#         #  x3 = (1 - a) * pix1[0] + a * contour[count+1][0]    # interpolate x coordinate
#         #   y3= (1 - a) * pix1[1] + a * contour[count+1][1]   # interpolate y coordinate
#         #   objmask3[int(x3),int(y3)] = 1 #add equidistant pixel to objmask3
#
#         objmask4 = np.dstack([objmask2.transpose()] * 3)  #change shape of objmask3
#
#         objmask2[objmask2 == 1] = classid  #Question: what does this do again?
#
#         #crop out the area outside the annotation using multiplication
#         imcrop_masked = np.multiply(objmask4, imcrop)
#
#         Image.fromarray((imcrop_masked).astype(np.uint8)).save('imcrop/class{}_obj{}.png'.format(classid, objID))
#         Image.fromarray(objmask2.astype(np.uint8)).save('mskcrop/classnew{}_obj{}.png'.format(classid, objID))

        #equidistant point
        #find bounding pixels, find point between neighboring pixels, add that pixel